#### Initial Segmentation and Mapping
- Segments images, enlarges them, and enhances their quality.
- Generates a **row mapping list** and **coordinates list** for all segmented images.
- Assigns a **row number** to each segmented image based on its position on the page for further processing.

In [21]:
import importlib as lib

import segmentation, save_and_load, mapping, identifications, modifications, generators, config

lib.reload(segmentation)
lib.reload(save_and_load)
lib.reload(mapping)
lib.reload(identifications)
lib.reload(modifications)
lib.reload(generators)
lib.reload(config)

<module 'config' from 'c:\\Users\\Admin\\Desktop\\Kernversion Backend\\config.py'>

In [2]:
from segmentation import extract_alphabets
from config import PATHS

pdf_path = PATHS['pdf_path']
initial_segmentation_folder = PATHS['initial_segmentation']
all_coordinates, all_row_mappings = extract_alphabets(pdf_path, initial_segmentation_folder)

Page 0 Row Mapping: [(1, 60, 68), (2, 95, 104), (3, 116, 125), (4, 145, 150), (5, 162, 169), (6, 187, 195), (7, 211, 238), (8, 252, 266), (9, 279, 285), (10, 294, 300), (11, 305, 320), (12, 325, 327), (13, 337, 350), (14, 355, 357), (15, 361, 367), (16, 374, 382), (17, 385, 387), (18, 391, 402), (19, 417, 431), (20, 444, 449), (21, 461, 467), (22, 477, 486), (23, 493, 505), (24, 522, 534), (25, 547, 552), (26, 558, 587), (27, 603, 617), (28, 629, 635), (29, 645, 651), (30, 657, 669), (31, 675, 678), (32, 688, 698), (33, 700, 707), (34, 712, 717), (35, 734, 736)]
Page 0 Coordinates: [(265, 734, 64, 5), (301, 713, 7, 7), (117, 713, 9, 9), (235, 712, 7, 9), (137, 705, 15, 5), (265, 704, 14, 5), (313, 700, 5, 8), (207, 692, 10, 12), (185, 692, 12, 13), (167, 692, 7, 12), (371, 691, 8, 12), (334, 691, 14, 13), (304, 691, 8, 12), (238, 691, 15, 13), (138, 691, 13, 11), (270, 690, 13, 11), (113, 688, 15, 16), (268, 676, 14, 4), (135, 675, 18, 6), (370, 667, 10, 4), (239, 663, 10, 12), (113, 6

#### User Input: Select Range of Row Numbers
- Accepts user input to specify the range of row numbers containing the **full composition**.
- Processes only the selected rows for further analysis and conversion.

In [3]:
# multiple_page.pdf
# first_row = 24
# last_row = 72

# # with_sanchari.pdf
# first_row = 16
# last_row = 73

# bilawal_dhamar.pdf
first_row = 2
last_row = 35

In [4]:
from generators import copy_images_in_row_range

working_composition_folder = PATHS['working_composition']

copy_images_in_row_range(initial_segmentation_folder, working_composition_folder, first_row, last_row)

#### Select the metadata (Raag, Taal, Lay etc.)

In [5]:
# raag_name = "Yaman"
# taal_name = "Ektaal"
# lay = "Vilambit"
# source_name = "hindustani-sangeet-paddhati-kramik-pustak-malika-part-2"


# raag_name = "Khamaj"
# taal_name = "Chautaal"
# lay = "Vilambit"
# source_name = "hindustani-sangeet-paddhati-kramik-pustak-malika-part-2"
# page_number = 147

raag_name = "Bilawal"
taal_name = "Dhamaar"
lay = "Vilambit"
source_name = "hindustani-sangeet-paddhati-kramik-pustak-malika-part-2"
page_number = 120

#### Show valid positions for the sam beat and take the input from user for sam beat

In [17]:
# from identifications import calculate_valid_sam_positions

# valid_positions = calculate_valid_sam_positions(taal_name)
# print(f"Valid Sam positions for {taal_name}: {valid_positions}")

In [ ]:
# sthayee_sam = 5
# antara_sam = 5

In [6]:
from save_and_load import save_composition_metadata

save_composition_metadata(raag_name, taal_name, lay, source_name=source_name, page_number=page_number)

In [ ]:
# from save_and_load import update_composition_metadata

# aabhog_sam = 7
# update_composition_metadata(aabhog_sam=aabhog_sam) 

In [ ]:
# from save_and_load import get_metadata_field

# taal = get_metadata_field(field="taal_name")
# sthayee_sam_beat = get_metadata_field(field="sthayee.sam_beat")

# print(f"Taal name: {taal}")
# print(f"First sam beat in Sthayee: {sthayee_sam_beat}")

Taal name: Ektaal
First sam beat in Sthayee: 5


In [ ]:
# from save_and_load import get_taal_data, get_taal_field

# taal_data = get_taal_data(taal)
# beat = get_taal_field(taal_name=taal, field_name="beat_count")

# print(f"Beat Count: {beat}")
# print(taal_data)

Beat Count: 12
{'beat_count': 12, 'divisions': [2, 2, 2, 2, 2, 2], 'vibhaag': ['X', '0', '2', '0', '3', '4'], 'time_signature': '12/8'}


In [ ]:
# from identifications import calculate_divisions_and_vibhaag

# # Calculate divisions and vibhaag
# new_divisions, new_vibhaag = calculate_divisions_and_vibhaag(taal, sthayee_sam_beat)

# # Define the beat count (size of the lists) from taal information
# beat_count = get_taal_field(taal_name=taal, field_name="beat_count")

# # Print the results
# print(f"\nTaal: {taal}")
# print(f"Total number of beats: {beat_count}")
# print(f"Sam (X) at beat: {sthayee_sam_beat}")
# print(f"New Divisions: {new_divisions}")
# print(f"New Vibhaag: {new_vibhaag}")


Taal: Ektaal
Total number of beats: 12
Sam (X) at beat: 5
New Divisions: [2, 2, 2, 2, 2, 2]
New Vibhaag: ['3', '4', 'X', '0', '2', '0']


#### Calculate Image Count for Each Row
- Counts the number of images in each row by processing the segmented image files.

In [7]:
from mapping import get_row_image_counts

# Get image counts for all rows
row_image_count = get_row_image_counts(working_composition_folder)

#### Identify Rows with Sam, Taalis, and Khali
- Creates a list of rows that could represent Sam, Taalis, and Khali based on the number of images in each row.
- Uses the given taal information to determine the expected number of images for these rows.

In [8]:
from identifications import get_sam_and_taalis_rows
from save_and_load import get_metadata_field

taal = get_metadata_field(field="taal_name")

sam_and_taalis_rows = get_sam_and_taalis_rows(row_image_count, taal_name=taal)
print(f"Rows with sam and taalis: ",sam_and_taalis_rows)

Rows with sam and taalis:  [10, 15, 16, 20, 22, 25, 28, 29, 2, 9]


#### User Correction: Update Identified Rows
- Allows the user to review and correct the automatically identified rows for Sam, Taalis, and Khali.
- Users can:
  - Add missing rows.
  - Remove incorrectly identified rows.
  - Update the list to ensure accuracy.
- Uses the corrected row numbers to create **subgroups**, where each subgroup includes:
  - A **swar row**.
  - Optional **kann swar**, **lyrics**, and **articulation rows**.

In [9]:
# sam_and_taalis_rows = [32, 38, 44, 52, 58, 65, 71] # multiple pages full composition - yaman-ektaal-vilambit

# sam_and_taalis_rows = [21, 26, 29, 35, 40, 44, 49, 53, 59, 64, 68, 72] # with_sanchari -> khamaj

sam_and_taalis_rows = [9, 15, 20, 25, 28, 34] # bilawal dhamaar 

#### Initial Subgroup Creation and Assigning Column Numbers
- Finds the first valid row in each subgroup.
- Assigns `extra` for images that are not part of the composition (noise and metadata).
- Assigns `column numbers` for valid images that are part of the composition.

In [10]:
from mapping import assign_column_numbers

subgroup_ranges = assign_column_numbers(all_row_mappings, first_row, sam_and_taalis_rows)
print("Subgroup Ranges:", subgroup_ranges)

Subgroup range: [2, 9]
First valid row in subgroup:  4
Subgroup range: [10, 15]
First valid row in subgroup:  10
Subgroup range: [16, 20]
First valid row in subgroup:  16
Subgroup range: [21, 25]
First valid row in subgroup:  22
Subgroup range: [26, 28]
First valid row in subgroup:  26
Subgroup range: [29, 34]
First valid row in subgroup:  29
Subgroup Ranges: [(4, 8), (10, 14), (16, 19), (22, 24), (26, 27), (29, 33)]


#### Update the subgroup if required
- When there is subgroup not having corresponding sam and taalis row (1st subgroup in most of the case)

In [11]:
from modifications import update_subgroups

updated_subgroups = update_subgroups(subgroup_ranges)

for start, end in updated_subgroups:
    print(f"Updated Subgroup range: [{start}, {end}]")

Updated Subgroup range: [4, 7]
Updated Subgroup range: [7, 8]
Updated Subgroup range: [10, 14]
Updated Subgroup range: [16, 19]
Updated Subgroup range: [22, 24]
Updated Subgroup range: [26, 27]
Updated Subgroup range: [29, 33]


In [12]:
# Assign back the updated subgroups
subgroup_ranges = updated_subgroups

for start, end in subgroup_ranges:
    print(f"Updated Subgroup range: [{start}, {end}]")

Updated Subgroup range: [4, 7]
Updated Subgroup range: [7, 8]
Updated Subgroup range: [10, 14]
Updated Subgroup range: [16, 19]
Updated Subgroup range: [22, 24]
Updated Subgroup range: [26, 27]
Updated Subgroup range: [29, 33]


#### Section Identification
- Identifies row numbers belonging to specific sections:
  - **Kann Swar**
  - **Swar**
  - **Lyrics**
  - **Articulation**
- Organizes rows into their respective sections for further processing.

In [13]:
from identifications import classify_rows_in_subgroups
from save_and_load import save_row_categories

row_categories = classify_rows_in_subgroups(subgroup_ranges)

# save the row numbers category wise
save_row_categories(row_categories)

Articulation Rows:  [12, 14, 17]
Kann Swar Rows:  [10, 16, 22]
Swar Rows:  [7, 11, 18, 23, 26]
Lyrics Rows:  [8, 13, 19, 24, 27]


#### User Correction and Selection
- Allows user to **correct** identified sections (**Kann Swar**, **Swar**, **Lyrics**, **Articulation**).
- Enables selection of **Sthayee**, **Antara**, **Sanchari** and **Aabhog** rows.
- Ensures accurate grouping and processing of rows based on user input.

In [ ]:
# row_categories = {
#     "articulation": [29, 31, 35, 37, 49, 51, 55, 57, 61, 64, 68, 70],
#     "kann_swar": [27, 33, 40, 47, 53, 59, 66],
#     "swar": [28, 34, 41, 48, 54, 60, 67],
#     "lyrics": [30, 36, 43, 50, 56, 62, 69],
#     "sthayee": 25,
#     "antara": 46
# }

# row_categories = {
#     "articulation": [],
#     "kann_swar": [18, 23, 31, 32, 41, 45, 46, 50, 56, 61, 65, 69],
#     "swar": [19, 24, 27, 33, 38, 42, 47, 51, 57, 62, 66, 70],
#     "lyrics": [20, 25, 28, 34, 39, 43, 48, 52, 58, 63, 67, 71],
#     "sthayee": 17,
#     "antara": 37,
#     "sanchari": 55
# }


# bilawal dhamaar
row_categories = {
    "articulation": [12, 14, 31, 33],
    "kann_swar": [4, 10, 16, 22, 29],
    "swar": [5, 7, 11, 18, 23, 26, 30],
    "lyrics": [6, 8, 13, 19, 24, 27, 32],
    "sthayee": 3,
    "antara": 21
}

# sanchari
# aabhog

In [15]:
from save_and_load import save_row_categories

save_row_categories(row_categories)

In [16]:
from save_and_load import load_row_categories

row_categories = load_row_categories()

# Extract specific row categories
articulation_rows = row_categories.get("articulation", [])
kann_swar_rows = row_categories.get("kann_swar", [])
swar_rows = row_categories.get("swar", [])
lyrics_rows = row_categories.get("lyrics", [])

# Print the results
print("Articulation Rows: ", articulation_rows)
print("Kann Swar Rows: ", kann_swar_rows)
print("Swar Rows: ", swar_rows)
print("Lyrics Rows: ", lyrics_rows)

Articulation Rows:  [12, 14, 31, 33]
Kann Swar Rows:  [4, 10, 16, 22, 29]
Swar Rows:  [5, 7, 11, 18, 23, 26, 30]
Lyrics Rows:  [6, 8, 13, 19, 24, 27, 32]


#### Get input for first sam beat of Sthayee, Antara, Sanchari and Aabhog (if applicable)

In [17]:
from save_and_load import load_row_categories

row_categories = load_row_categories()

sthayee = row_categories.get("sthayee")
antara = row_categories.get("antara")
sanchari = row_categories.get("sanchari")
aabhog = row_categories.get("aabhog")


from save_and_load import update_composition_metadata

# Dictionary to store section beats (only for sections that exist)
section_beats = {}
if sthayee is not None:
    section_beats['sthayee_sam_beat'] = 5
if antara is not None:
    section_beats['antara_sam_beat'] = 1
if sanchari is not None:
    section_beats['sanchari_sam_beat'] = -1
if aabhog is not None:
    section_beats['aabhog_sam_beat'] = -1

# Single update call with all non-None sections
if section_beats:
    update_composition_metadata(**section_beats)

#### Initial List Creation and Mapping
- Creates lists for each subgroup:
  - **Kann Swar**
  - **Swar**
  - **Swar Articulation Checks**
  - **Lyrics**
  - **Lyrics Articulation Checks**
- Stores image file paths in the respective lists.

#### Kann Swar and Swar Mapping
- **Case 1: Explicit Kann Swar Row**  
  - Maps **Kann Swar** to its corresponding **Swar** directly.
- **Case 2: Hidden Kann Swar in Swar Row**  
  - Identifies **Kann Swar** images present in the **Swar** row and moves them to the **Kann Swar** list.
  - Detects and segments composite images containing both **Kann Swar** and **Swar**.
  - Stores the segmented **Kann Swar** image path in the **Kann Swar** list.
  - Stores the segmented **Swar** image path in the **Swar** list.

#### Articulation Mapping
- Maps **Swar Articulation** rows to their corresponding **Swar**.
- Maps **Lyrics Articulation** rows to their corresponding **Lyrics**.

In [22]:
from generators import generate_lists_in_subgroups
import os
from filename_utils import get_image_details_with_path
from collections import defaultdict
from save_and_load import get_taal_field

# Load all image filenames and extract their information
image_files = os.listdir(working_composition_folder)
image_info = [get_image_details_with_path(f, working_composition_folder) for f in image_files]
image_info = [info for info in image_info if info is not None]

# Organize images by row and column
row_col_images = defaultdict(lambda: defaultdict(list))
for info in image_info:
    page_num, row_num, col_num, x, y, width, height, image_path = info
    row_col_images[row_num][col_num].append((x, y, width, height, image_path))

# get beat count for given taal
beat_count = get_taal_field(taal, field_name="beat_count")

# Function to generate list of lists for all sections in each subgroup
subgroups = generate_lists_in_subgroups(subgroup_ranges, row_col_images, beat_count)

In [23]:
from save_and_load import save_lists_in_subgroups

save_lists_in_subgroups(subgroups)

In [24]:
from save_and_load import load_lists_in_subgroups

subgroups = load_lists_in_subgroups()

# Print the results for each subgroup
for subgroup_range, results in subgroups.items():
    print(f"Subgroup Range: {subgroup_range}")
    print(f"Kann Swar List: {results['kann_swar_list']}")
    print(f"Swar List: {results['swar_list']}")
    print(f"Swar Articulation Checks: {results['swar_articulation_checks']}")
    print(f"Lyrics List: {results['lyrics_list']}")
    print(f"Lyrics Articulation Checks: {results['lyrics_articulation_checks']}")
    print("-" * 80)

Subgroup Range: (4, 7)
Kann Swar List: [[], [], [], [], [], [], [], [], [], [], [], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row4_col12_x400_y145_w7_h10.png'], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row4_col13_x434_y145_w9_h10.png'], []]
Swar List: [[], [], [], [], [], [], [], [], [], [], [], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row5_col12_x406_y162_w10_h12.png'], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row5_col13_x439_y162_w10_h14.png'], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row5_col14_x472_y166_w9_h5.png']]
Swar Articulation Checks: [[], [], [], [], [], [], [], [], [], [], [], False, False, False]
Lyrics List: [[], [], [], [], [], [], [], [], [], [], [], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row6_col12_x405_y188_w11_h15.png'], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row6_col13_x436_y188_w14_h12.png'], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row6_col14_x473_y187_w8_h1

#### Meend Segmentation and List Creation
- Segments composite **Meend** and **Kann Swar** images.
- Stores the **Kann Swar** image in its correct position in the **Kann Swar List** based on coordinates.
- Creates a separate **Meend List** to store:
  - **Start** and **end positions** of each **Meend** corresponding to the **Swar** row.
- Detects **Meend** in the **Kann Swar** row.
- If **Meend** is present:
    - Update the **Meend List**
    - Remove the image path from **Kann Swar List**

In [25]:
from generators import update_kann_swar_and_generate_meend_lists

update_kann_swar_and_generate_meend_lists()

In [26]:
subgroups = load_lists_in_subgroups()

# Print the updated results for each subgroup
for subgroup_range, results in subgroups.items():
    print(f"Subgroup Range: {subgroup_range}")
    print(f"Kann Swar List: {results['kann_swar_list']}")
    print(f"Meend List: {results['meend_list']}")
    print(f"Swar List: {results['swar_list']}")
    print(f"Lyrics List: {results['lyrics_list']}")
    print(f"Swar Articulation Checks: {results['swar_articulation_checks']}")
    print(f"Lyrics Articulation Checks: {results['lyrics_articulation_checks']}")
    print("-" * 80)

Subgroup Range: (4, 7)
Kann Swar List: [[], [], [], [], [], [], [], [], [], [], [], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row4_col12_x400_y145_w7_h10.png'], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row4_col13_x434_y145_w9_h10.png'], []]
Meend List: ['', '', '', '', '', '', '', '', '', '', '', '', '', '']
Swar List: [[], [], [], [], [], [], [], [], [], [], [], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row5_col12_x406_y162_w10_h12.png'], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row5_col13_x439_y162_w10_h14.png'], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row5_col14_x472_y166_w9_h5.png']]
Lyrics List: [[], [], [], [], [], [], [], [], [], [], [], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row6_col12_x405_y188_w11_h15.png'], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row6_col13_x436_y188_w14_h12.png'], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row6_col14_x473_y187_w8_h12.png']]
Swar Articulat

#### Final Segmentation and List Finalization
- Performs further segmentation to finalize lists for each subgroup.

#### Articulation Detection and Segmentation
- Detects and segments **articulation** from:
  - **Swar Row**: Updates the **Swar Articulation Checks** list.
  - **Lyrics Row**: Updates the **Lyrics Articulation Checks** list.

#### Word Segmentation
- Applies **word segmentation** to:
  - **Swar Row**: For images corresponding to **Swar Articulation Checks**.
  - **Lyrics Row**: For images corresponding to **Lyrics Articulation Checks**.

In [27]:
from modifications import finalize_segmentation_and_lists

finalize_segmentation_and_lists()

In [28]:
subgroups = load_lists_in_subgroups()

# Print the updated results for each subgroup
for subgroup_range, results in subgroups.items():
    print(f"Subgroup Range: {subgroup_range}")
    print(f"Kann Swar List: {results['kann_swar_list']}")
    print(f"Meend List: {results['meend_list']}")
    print(f"Swar List: {results['swar_list']}")
    print(f"Lyrics List: {results['lyrics_list']}")
    print(f"Swar Articulation Checks: {results['swar_articulation_checks']}")
    print(f"Lyrics Articulation Checks: {results['lyrics_articulation_checks']}")
    print("-" * 80)

Subgroup Range: (4, 7)
Kann Swar List: [[], [], [], [], [], [], [], [], [], [], [], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row4_col12_x400_y145_w7_h10.png'], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row4_col13_x434_y145_w9_h10.png'], []]
Meend List: ['', '', '', '', '', '', '', '', '', '', '', '', '', '']
Swar List: [[], [], [], [], [], [], [], [], [], [], [], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row5_col12_x406_y162_w10_h12.png'], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row5_col13_x439_y162_w10_h14.png'], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row5_col14_x472_y166_w9_h5.png']]
Lyrics List: [[], [], [], [], [], [], [], [], [], [], [], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row6_col12_x405_y188_w11_h15.png'], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row6_col13_x436_y188_w14_h12.png'], ['Analysis\\bilawal_dhamaar_2\\working_composition\\0_row6_col14_x473_y187_w8_h12.png']]
Swar Articulat

#### Prediction for Kann Swar and Swar Rows
- Predicts results for:
  - **Kann Swar Row Images**: Uses the **Kann Swar List** for predictions.
  - **Swar Row Images**: Uses the **Swar List** for predictions.
- Generates predictions for each subgroup based on the respective lists.

In [ ]:
# from save_and_load import load_classes

# classes = load_classes()

# print(classes[0])

saa


In [29]:
from generators import generate_predictions

subgroups = load_lists_in_subgroups()
predicted_results = generate_predictions(subgroups)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step


In [30]:
from save_and_load import save_predictions

save_predictions(predicted_results)

In [31]:
from save_and_load import load_predictions

def print_predictions(predictions):
    """Prints results with original formatting"""
    for subgroup_range, results in predictions.items():
        print(f"\nSubgroup Range: {subgroup_range}")
        print(f"Predicted Kann Swar List: {results['kann_swar']}")
        print(f"Predicted Swar List: {results['swar']}") 
        print("-" * 80)

predictions = load_predictions()

if predictions:
    print_predictions(predictions)


Subgroup Range: (4, 7)
Predicted Kann Swar List: [[], [], [], [], [], [], [], [], [], [], [], ['pa'], ['dha'], []]
Predicted Swar List: [[], [], [], [], [], [], [], [], [], [], [], ['dha'], ['ga'], ['-']]
--------------------------------------------------------------------------------

Subgroup Range: (7, 8)
Predicted Kann Swar List: [[], [], ["saa'"], ['dha'], [], [], [], [], ['dha'], [], [], ["saa'"], [], []]
Predicted Swar List: [['pa'], ['-'], ['ni'], ['ni'], ["saa'"], ['-'], ['-'], ['ni'], ['ni'], ["saa'"], ['-'], ["re'"], ["saa'"], ['-']]
--------------------------------------------------------------------------------

Subgroup Range: (10, 14)
Predicted Kann Swar List: [["saa'"], [], [], [], ['pa'], ['pa'], [], ['ma'], [], [], [], [], [], []]
Predicted Swar List: [['dha'], ['dha,', 'ni-'], ['pa'], ['-'], ['ga'], ['ga'], ['ma#', 're'], ['ga'], ['ma'], ['pa'], ['saa', 'saa'], ['ga'], ['re'], ['-']]
--------------------------------------------------------------------------------

S

In [ ]:
# from identifications import calculate_divisions_and_vibhaag
# from save_and_load import get_metadata_field, get_taal_field

# # taal = get_metadata_field(field='taal_name')
# # sthayee_sam = get_metadata_field(field='sthayee.sam_beat')

# taal = "Ektaal"
# sthayee_sam_beat = 7

# # Calculate divisions and vibhaag
# new_divisions, new_vibhaag = calculate_divisions_and_vibhaag(taal, sthayee_sam_beat)

# # Define the beat count (size of the lists) from taal information
# beat_count = get_taal_field(taal, field_name="beat_count")

# # Print the results
# print(f"\nTaal: {taal}")
# print(f"Total number of beats: {beat_count}")
# print(f"Sam (X) at beat: {sthayee_sam_beat}")
# print(f"New Divisions: {new_divisions}")
# print(f"New Vibhaag: {new_vibhaag}")



Taal: Ektaal
Total number of beats: 12
Sam (X) at beat: 7
New Divisions: [2, 2, 2, 2, 2, 2]
New Vibhaag: ['0', '3', '4', 'X', '0', '2']


#### Separate Prediction for Sthayee, Antara, Sanchari and Aabhog
- Processes musical predictions separately for each raga section:
  - **Sthayee**
  - **Antara**  
  - **Sanchari** 
  - **Aabhog** 

#### Flattening Process
- Converts different beat cycles into linear sequences:
  - **Swar List**: 
  - **Kann Swar List**: 
- Maintains synchronization between swar and kann_swar sequences

In [32]:
from save_and_load import load_predictions
from save_and_load import load_row_categories
from modifications import categorize_flatten_predictions

# def print_categorized_predictions(categorized):
#     """Prints the categorized predictions"""
#     for category, data in categorized.items():
#         if data:  # Only print if category exists
#             print(f"\n{category.upper()}:")
#             print("Swar:", data["swar"])
#             print("Kann Swar:", data["kann_swar"])


predictions = load_predictions()
row_categories = load_row_categories()

if predictions:
        categorized_flatten = categorize_flatten_predictions(predictions, row_categories)
        # print_categorized_predictions(categorized_flatten)

In [33]:
from save_and_load import save_categorized_flatten_predictions

save_categorized_flatten_predictions(categorized_flatten)

#### Generate metadata header for the kern

In [34]:
from save_and_load import get_metadata_field
from generators import generate_metadata_header

raag = get_metadata_field(field='raag_name')
taal = get_metadata_field(field='taal_name')
lay = get_metadata_field(field='lay')

metadata_header_text, metadata_header_kern = generate_metadata_header(raag, taal, lay)

# print(metadata_header_text)

#### Sthayee Conversion

In [35]:
from generators import generate_kern
from save_and_load import load_categorized_flatten_predictions
from identifications import calculate_divisions_and_vibhaag
from save_and_load import get_metadata_field, get_taal_field

taal = get_metadata_field(field='taal_name')
sthayee_sam_beat = get_metadata_field(field='sthayee.sam_beat')

# Calculate divisions and vibhaag for sthayee
sthayee_division, sthayee_vibhaag = calculate_divisions_and_vibhaag(taal, sthayee_sam_beat)

# Number of beats in one cycle of given taal
beat_count = get_taal_field(taal, field_name="beat_count")

sthayee_predictions = load_categorized_flatten_predictions("sthayee")
sthayee_kann_swar = sthayee_predictions["kann_swar"]
sthayee_swar = sthayee_predictions["swar"]

sthayee_kern_text, sthayee_kern = generate_kern(sthayee_kann_swar, sthayee_swar, sthayee_division, beat_count)

sthayee_header = "!! Sthayee"

# print(metadata_header_text)
# print(sthayee_header)
# print(sthayee_kern_text)

#### Sthayee to Antara Transition

In [36]:
from generators import generate_transition

# get sam beat for antara
antara_sam_beat = get_metadata_field(field='antara.sam_beat')

if antara_sam_beat is not None:

    # Calculate divisions and vibhaag for antara
    antara_division, antara_vibhaag = calculate_divisions_and_vibhaag(taal, antara_sam_beat)

    # Generate transition from sthayee to antara
    sthayee_antara_transition_text, sthayee_antara_transition_kern = generate_transition(sthayee_kern, sthayee_vibhaag, antara_vibhaag)

    sthayee_antara_transition_header = "!! Sthayee to Antara Transition"

    # print(sthayee_kern_text)
    # print(sthayee_antara_transition_text)

#### Antara Conversion

In [37]:
from generators import generate_kern
from save_and_load import load_categorized_flatten_predictions

if antara_sam_beat is not None:

    antara_predictions = load_categorized_flatten_predictions("antara")
    antara_kann_swar = antara_predictions["kann_swar"]
    antara_swar = antara_predictions["swar"]

    antara_kern_text, antara_kern = generate_kern(antara_kann_swar, antara_swar, antara_division, beat_count)

    antara_header = "!! Antara"
    # print(antara_kern_text)

#### Antara to Sanchari Transition (if exists)

In [38]:
from generators import generate_transition

# get sam beat for antara
sanchari_sam_beat = get_metadata_field(field='sanchari.sam_beat')

if sanchari_sam_beat is not None:

    # Calculate divisions and vibhaag for sanchari
    sanchari_division, sanchari_vibhaag = calculate_divisions_and_vibhaag(taal, sanchari_sam_beat)

    # Generate transition from sthayee to antara
    antara_sanchari_transition_text, antara_sanchari_transition_kern = generate_transition(antara_kern, antara_vibhaag, sanchari_vibhaag)

    antara_sanchari_transition_header = "!! Antara to Sanchari Transition"
    # print(sthayee_antara_transition_text)

#### Sanchari Conversion

In [39]:
from generators import generate_kern
from save_and_load import load_categorized_flatten_predictions

if sanchari_sam_beat is not None:

    sanchari_predictions = load_categorized_flatten_predictions("sanchari")
    sanchari_kann_swar = sanchari_predictions["kann_swar"]
    sanchari_swar = sanchari_predictions["swar"]

    sanchari_kern_text, sanchari_kern = generate_kern(sanchari_kann_swar, sanchari_swar, sanchari_division, beat_count)

    sanchari_header = "!! Sanchari"
    # print(sanchari_kern_text)

#### Sanchari to Aabhog Transition (if exists)

In [40]:
from generators import generate_transition

# get sam beat for antara
aabhog_sam_beat = get_metadata_field(field='aabhog.sam_beat')

if aabhog_sam_beat is not None:

    # Calculate divisions and vibhaag for sanchari
    aabhog_division, aabhog_vibhaag = calculate_divisions_and_vibhaag(taal, aabhog_sam_beat)

    # Generate transition from sthayee to antara
    sanchari_aabhog_transition_text, sanchari_aabhog_transition_kern = generate_transition(sanchari_kern, sanchari_vibhaag, aabhog_vibhaag)

    sanchari_aabhog_transition_header = "!! Sanchari to Aabhog Transition"
    # print(sanchari_aabhog_transition_text)

#### Aabhog Conversion

In [41]:
from generators import generate_kern
from save_and_load import load_categorized_flatten_predictions

if aabhog_sam_beat is not None:

    aabhog_predictions = load_categorized_flatten_predictions("aabhog")
    aabhog_kann_swar = aabhog_predictions["kann_swar"]
    aabhog_swar = aabhog_predictions["swar"]

    aabhog_kern_text, aabhog_kern = generate_kern(aabhog_kann_swar, aabhog_swar, aabhog_division, beat_count)

    aabhog_header = "!! Aabhog"
    # print(aabhog_kern_text)

#### Print full kern code

In [ ]:
print(metadata_header_text)

if sthayee_sam_beat:
    print(sthayee_header)
    print(sthayee_kern_text)

if antara_sam_beat:
    print(sthayee_antara_transition_header)
    print(sthayee_antara_transition_text)

    print(antara_header)
    print(antara_kern_text)

if sanchari_sam_beat:

    print(antara_sanchari_transition_header)
    print(antara_sanchari_transition_text)

    print(sanchari_header)
    print(sanchari_kern_text)

if aabhog_sam_beat:

    print(sanchari_aabhog_transition_header)
    print(sanchari_aabhog_transition_text)

    print(aabhog_header)
    print(aabhog_kern_text)

!!!raag: Bilawal
!!!taal: Dhamaar
!!!lay: Vilambit
!!!source: hindustani-sangeet-paddhati-kramik-pustak-malika-part-2
!!!page: 120
**kern
*M14/8
*MM60
*Isitar
*clefG2
*c:
!! Sthayee
==1
4ryy
4ryy
4ryy
4ryy
=
4ryy
4ryy
4ryy
4ryy
4ryy
=
4ryy
4ryy
=
gq
4a
aq
2e
==2
2g
ccq
4b
aq
4b
=
2.cc
4b
aq
4b
=
2cc
=
ccq
4dd
2cc
==3
ccq
4a
8A
8b-
2g
=
gq
4e
gq
4e
8f#
8d
fq
4e
4f
=
4g
8c
8c
=
4e
2d
==4
2.c
4c
=
bq
4cc
2cc
ccq
2a
=
4b-
4g
=
gq
4a
aq
2e
!! Sthayee to Antara Transition
gq
4a
aq
2e
=
2g
ccq
4b
aq
4b
=
!! Antara
==1
bq
2.cc
ccq
4b
aq
4b
=
2cc
=
2cc
4cc
=
bq
4cc
4dd
2cc
==2
bq
4cc
2ee
4dd
2.ee
=
=
4ff
ff#q
2.dd
=
bq
4cc
4dd
4cc
==3
4cc
cq
4a
8cc
8b
4dd
4cc
=
4cc
cq
4a
=
8b-
8gg
gq
4a
aq
2e
=
